# Import packages and setup

In [2]:
%matplotlib inline
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

# Read in IgBLAST BCR data

In [3]:
source_folder = "/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/IgBLAST_output_BCR"

file_list = [x for x in os.listdir(f'{source_folder}') if x.endswith('pass.tsv')]

file_dfs = []

for file in file_list:    
    file_df = pd.read_csv(f'{source_folder}/{file}', sep='\t')
    file_df['pool'] =  file.split("_")[0]
    file_dfs.append(file_df)
    
bcr_df = pd.concat(file_dfs)

In [4]:
bcr_df.pool.nunique()

68

In [5]:
len(bcr_df)

229108

# Read in sample, pool and batch info

In [6]:
batch_info = pd.read_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/1_Metadata/slemap_cellranger_to_pool.txt", sep = "\t", header = None)
batch_info.columns = ["sample", "pool", "scSeq_batch"]

In [7]:
bcr_df = pd.merge(bcr_df, batch_info)

In [8]:
bcr_df["contig"] = bcr_df["sequence_id"].str.split("_", n = 1).str[1]
bcr_df["chain"] = np.where(bcr_df['locus'].str.contains('IGH'), "Heavy" , "Light")
bcr_df["sample_cell_id"] = bcr_df["sample"] + "_" + bcr_df["cell_id"]

In [9]:
chain_counts_bcr = bcr_df.groupby(['pool','chain'])['sample_cell_id'].agg(number_chains = "value_counts").reset_index()
chain_counts_heavy = chain_counts_bcr.query("chain == 'Heavy'").copy()

In [10]:
chain_counts_heavy

,pool,chain,sample_cell_id,number_chains
0,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,2
1,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,2
2,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,2
3,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,2
4,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,2
...,...,...,...,...
221288,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1
221289,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1
221290,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1
221291,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1


In [11]:
chain_counts_heavy["BCR_doublet"] = np.where(chain_counts_heavy['number_chains'] == 1, "No" , "Yes")

In [12]:
chain_counts_heavy[["sample_cell_id", "pool", "BCR_doublet"]]

,sample_cell_id,pool,BCR_doublet
0,cellranger700_multi_dbbf8652dc66faa6a536508357...,LDP02,Yes
1,cellranger700_multi_dbbf8652dc66faa6a536508357...,LDP02,Yes
2,cellranger700_multi_dbbf8652dc66faa6a536508357...,LDP02,Yes
3,cellranger700_multi_dbbf8652dc66faa6a536508357...,LDP02,Yes
4,cellranger700_multi_dbbf8652dc66faa6a536508357...,LDP02,Yes
...,...,...,...
221288,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No
221289,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No
221290,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No
221291,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No


In [13]:
chain_counts_heavy["barcode"] = chain_counts_heavy["sample_cell_id"].str.rsplit("_", n=1).str[1]

chain_counts_heavy.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/bcr_doublet_barcodes.csv", index = False)

In [14]:
chain_counts_heavy.BCR_doublet.value_counts()

BCR_doublet
No     105952
Yes      1536
Name: count, dtype: int64

In [15]:
has_single_bcr = chain_counts_heavy.query("number_chains == 1").copy()
has_single_bcr["VDJ"] = "BCR"

# Read in IgBLAST TCR data

In [16]:
source_folder = "/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/IgBLAST_output_TCR"

file_list = [x for x in os.listdir(f'{source_folder}') if x.endswith('pass.tsv')]

file_dfs = []

for file in file_list:    
    file_df = pd.read_csv(f'{source_folder}/{file}', sep='\t')
    file_df['pool'] =  file.split("_")[0]
    file_dfs.append(file_df)
    
tcr_df = pd.concat(file_dfs)

In [17]:
tcr_df.pool.nunique()

69

In [18]:
len(tcr_df)

1177627

# Read in sample, pool and batch info

In [19]:
tcr_df = pd.merge(tcr_df, batch_info)

In [20]:
tcr_df["contig"] = tcr_df["sequence_id"].str.split("_", n = 1).str[1]
tcr_df["chain"] = np.where(tcr_df['locus'].str.contains('TRB'), "Beta" , "Alpha")
tcr_df["sample_cell_id"] = tcr_df["sample"] + "_" + tcr_df["cell_id"]

In [21]:
tcr_df.locus.value_counts()

locus
TRB    629316
TRA    548311
Name: count, dtype: int64

In [22]:
tcr_df.chain.value_counts()

chain
Beta     629316
Alpha    548311
Name: count, dtype: int64

In [23]:
chain_counts_tcr = tcr_df.groupby(['pool', 'chain'])['sample_cell_id'].agg(number_chains = "value_counts").reset_index()
chain_counts_beta = chain_counts_tcr.query("chain == 'Beta'").copy()

In [24]:
chain_counts_beta["tcr_doublet"] = np.where(chain_counts_beta['number_chains'] == 1, "No" , "Yes")

In [25]:
chain_counts_beta[["sample_cell_id", "pool", "tcr_doublet"]]

,sample_cell_id,pool,tcr_doublet
2708,cellranger700_multi_21212877c397b6975eaf35c723...,LDP01,Yes
2709,cellranger700_multi_21212877c397b6975eaf35c723...,LDP01,Yes
2710,cellranger700_multi_21212877c397b6975eaf35c723...,LDP01,Yes
2711,cellranger700_multi_21212877c397b6975eaf35c723...,LDP01,Yes
2712,cellranger700_multi_21212877c397b6975eaf35c723...,LDP01,Yes
...,...,...,...
1091760,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No
1091761,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No
1091762,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No
1091763,cellranger700_multi_47347f8bd01f6c23c63871b334...,LDP69,No


In [26]:
chain_counts_beta.tcr_doublet.value_counts()

tcr_doublet
No     559978
Yes     34669
Name: count, dtype: int64

In [27]:
chain_counts_beta["barcode"] = chain_counts_beta["sample_cell_id"].str.rsplit("_", n=1).str[1]

In [28]:
chain_counts_beta.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/tcr_doublet_barcodes.csv", index = False)

In [29]:
has_single_tcr = chain_counts_beta.query("number_chains == 1").copy()
has_single_tcr["VDJ"] = "TCR"

# Get TCR/BCR doublets

In [30]:
has_trb = list(chain_counts_beta.sample_cell_id)

In [31]:
has_igh = list(chain_counts_heavy.sample_cell_id)

In [32]:
len(list(set(has_trb) & set(has_igh)))

15262

In [33]:
tcr_bcr_doublet = pd.DataFrame(list(set(has_trb) & set(has_igh)))

In [34]:
tcr_bcr_doublet = tcr_bcr_doublet.rename({0:"sample_cell_id"}, axis = 1)

In [35]:
tcr_bcr_doublet["sample"] = tcr_bcr_doublet["sample_cell_id"].str.rsplit("_", n=1).str[0]
tcr_bcr_doublet["barcode"] = tcr_bcr_doublet["sample_cell_id"].str.rsplit("_", n=1).str[1]

In [36]:
tcr_bcr_doublet = pd.merge(tcr_bcr_doublet, batch_info)

In [37]:
tcr_bcr_doublet.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/tcr_bcr_doublet_barcodes.csv", index = False)

In [38]:
tcr_bcr_doublet

,sample_cell_id,sample,barcode,pool,scSeq_batch
0,cellranger700_multi_4aa86417eee207b8497d71521f...,cellranger700_multi_4aa86417eee207b8497d71521f...,CAGAGAGAGGGCACTA-1,LDP13,3
1,cellranger700_multi_51aaa01a9a405602a00f159c1d...,cellranger700_multi_51aaa01a9a405602a00f159c1d...,CATATGGAGATCGATA-1,LDP44,6
2,cellranger700_multi_850906bde9153135a2abd77d02...,cellranger700_multi_850906bde9153135a2abd77d02...,ACTTACTGTACCCAAT-1,LDP27,5
3,cellranger700_multi_c9d76f90bbd437718946517c9e...,cellranger700_multi_c9d76f90bbd437718946517c9e...,CTAGAGTGTTGATTCG-1,LDP17,4
4,cellranger700_multi_46a979d5d5e827d4e8f0b90b33...,cellranger700_multi_46a979d5d5e827d4e8f0b90b33...,TCAGATGGTTCCGTCT-1,LDP42,6
...,...,...,...,...,...
15257,cellranger700_multi_0f2e6fd46960ceef6a6023494e...,cellranger700_multi_0f2e6fd46960ceef6a6023494e...,GCAATCAAGCTGAACG-1,LDP21,4
15258,cellranger700_multi_800e1312938cc9ab17a0dec453...,cellranger700_multi_800e1312938cc9ab17a0dec453...,TGATTTCAGCTTTGGT-1,LDP40,6
15259,cellranger700_multi_47347f8bd01f6c23c63871b334...,cellranger700_multi_47347f8bd01f6c23c63871b334...,ACAGCCGGTCCTGCTT-1,LDP69,10
15260,cellranger700_multi_d00ddfc9ace09586f1ac108fca...,cellranger700_multi_d00ddfc9ace09586f1ac108fca...,GAAGCAGCAAATCCGT-1,LDP25,5


# Get cells with a single heavy or beta chain
Not checking for paired receptors yet

In [39]:
has_single_tcr = has_single_tcr.drop("tcr_doublet",  axis =1)
has_single_tcr

,pool,chain,sample_cell_id,number_chains,barcode,VDJ
2956,LDP01,Beta,cellranger700_multi_21212877c397b6975eaf35c723...,1,AAACCTGAGATGTGTA-1,TCR
2957,LDP01,Beta,cellranger700_multi_21212877c397b6975eaf35c723...,1,AAACCTGCACCGGAAA-1,TCR
2958,LDP01,Beta,cellranger700_multi_21212877c397b6975eaf35c723...,1,AAACCTGGTTACGGAG-1,TCR
2959,LDP01,Beta,cellranger700_multi_21212877c397b6975eaf35c723...,1,AAACCTGTCCTAAGTG-1,TCR
2960,LDP01,Beta,cellranger700_multi_21212877c397b6975eaf35c723...,1,AAACCTGTCCTCATTA-1,TCR
...,...,...,...,...,...,...
1091760,LDP69,Beta,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCAGTTTAGCTG-1,TCR
1091761,LDP69,Beta,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCATCGCAAACT-1,TCR
1091762,LDP69,Beta,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCATCTCGTTTA-1,TCR
1091763,LDP69,Beta,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCATCTTAGCCC-1,TCR


In [40]:
has_single_bcr = has_single_bcr.drop("BCR_doublet",  axis =1)
has_single_bcr

,pool,chain,sample_cell_id,number_chains,barcode,VDJ
57,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,1,AAACCTGAGACCGGAT-1,BCR
58,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,1,AAACCTGAGGGAACGG-1,BCR
59,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,1,AAACCTGCAAACGCGA-1,BCR
60,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,1,AAACCTGCAATCTGCA-1,BCR
61,LDP02,Heavy,cellranger700_multi_dbbf8652dc66faa6a536508357...,1,AAACCTGGTAGAAGGA-1,BCR
...,...,...,...,...,...,...
221288,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCAAGACAAGCC-1,BCR
221289,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCAAGCTACCTA-1,BCR
221290,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCAAGTGAAGTT-1,BCR
221291,LDP69,Heavy,cellranger700_multi_47347f8bd01f6c23c63871b334...,1,TTTGTCAGTACCCAAT-1,BCR


In [41]:
has_single_bcr_tcr = pd.concat([has_single_bcr, has_single_tcr])

In [42]:
has_single_bcr_or_tcr = has_single_bcr_tcr.copy()

has_single_bcr_or_tcr = has_single_bcr_or_tcr[~has_single_bcr_or_tcr.sample_cell_id.isin(list(set(has_trb) & set(has_igh)))]

In [43]:
has_single_bcr_or_tcr.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/has_tcr_bcr_barcodes.csv", index = False)

In [72]:
check_a = list(has_single_bcr_or_tcr.sample_cell_id)

In [73]:
check_b = list(tcr_bcr_doublet.sample_cell_id)

In [74]:
len(list(set(check_a) & set(check_b)))

0

# Get single heavy BCR chain info for simple annotation

In [44]:
has_single_bcr_barcode = has_single_bcr_or_tcr.query("VDJ == 'BCR'")['sample_cell_id'].copy()

In [45]:
bcr_df_heavy_single_bcr  = bcr_df.query("(sample_cell_id in @has_single_bcr_barcode) & (locus == 'IGH')").copy()
bcr_df_heavy_single_bcr["sample"] = bcr_df_heavy_single_bcr["sample_cell_id"].str.rsplit("_", n=1).str[0]
bcr_df_heavy_single_bcr["barcode"] = bcr_df_heavy_single_bcr["sample_cell_id"].str.rsplit("_", n=1).str[1]

In [48]:
bcr_df_heavy_single_bcr = bcr_df_heavy_single_bcr.dropna(subset = ["sequence_alignment", "germline_alignment"]).copy()


bcr_df_heavy_single_bcr['mut_freq_v'] = bcr_df_heavy_single_bcr.apply(lambda row : get_mut_freq_v(row['sequence_alignment'],
                 row['germline_alignment']), axis = 1)

In [49]:
bcr_df_heavy_single_bcr.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/users/cs54/final_data/2_BCR_TCR/1_IgBLAST_assignment/bcr_df_heavy_single_bcr.csv", index = False)